<a href="https://colab.research.google.com/github/MitziVite/machine_learning/blob/main/notebooks/starter_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes.head()

In [ ]:
bikes["total_rentals"] = bikes["casual"] + bikes["registered"]

bikes["dteday"] = pd.to_datetime(bikes["dteday"])
bikes["day"] = bikes["dteday"].dt.day
bikes["month"] = bikes["dteday"].dt.month
bikes["year"] = bikes["dteday"].dt.year

features = [
    "hr", "temp_c", "feels_like_c", "hum", "windspeed",
    "weathersit", "season", "holiday", "workingday",
    "day", "month", "year"
]

X = bikes[features]
y = bikes["total_rentals"]


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.01), loss="mse", metrics=["mae"])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    verbose=1
)

# Evaluation
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f}")

# Predictions
bikes["predicted_total"] = model.predict(scaler.transform(X))

# Save only the first 384 predictions as 'total_rentals'
bikes["predicted_total"].head(384).to_csv("bike_predictions.csv", index=False, header=['total_rentals'])
print("✅ CSV export complete!")

# Task
Improve the model's performance in the notebook "/content/module04_biking_grading.ipynb" by iterating on the model architecture, hyperparameters, feature engineering, and regularization techniques, evaluating the results using the provided metrics (Within 5%, Within 10%, Within 20%, R^2, RMSE, MAE, MedAE) after each change, and summarizing the changes and results.

## Revisar la arquitectura del modelo

### Subtask:
Experimentar con un número diferente de capas densas y neuronas en cada capa.


**Reasoning**:
Modify the model architecture with a different number of dense layers and neurons, keeping the input shape and the output layer the same, and then train and evaluate the model.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mae"])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluation
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f}")

# Predictions
bikes["predicted_total"] = model.predict(scaler.transform(X))

# Save only the first 384 predictions as 'total_rentals'
bikes["predicted_total"].head(384).to_csv("bike_predictions.csv", index=False, header=['total_rentals'])
print("✅ CSV export complete!")

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 97607.5703 - mae: 216.3812 - val_loss: 55965.7500 - val_mae: 161.7323
Epoch 2/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 51061.4766 - mae: 152.2376 - val_loss: 39970.6016 - val_mae: 127.8822
Epoch 3/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 37697.1328 - mae: 125.5851 - val_loss: 30557.0488 - val_mae: 114.4259
Epoch 4/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 28406.2695 - mae: 108.2564 - val_loss: 22247.0840 - val_mae: 94.4678
Epoch 5/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 21021.8828 - mae: 92.9941 - val_loss: 19584.2285 - val_mae: 89.4839
Epoch 6/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 17121.4648 - mae: 83.3580 - val_loss: 14845.8818 - val_mae: 76.5877
Epoch 7/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 14447.5713 - mae: 76.5932 - val_loss: 13144.9307 - val_mae: 72.5155
Epoch 8/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 12915.7188 - mae: 72.1495 - val_